In [1]:
%%bash
pip install --upgrade pip
pip install smart_open minecart
pip install textract-trp

  Using cached pip-20.2.3-py2.py3-none-any.whl (1.5 MB)
  Attempting uninstall: pip
    Found existing installation: pip 20.0.2
    Uninstalling pip-20.0.2:
      Successfully uninstalled pip-20.0.2
  Created wheel for smart-open: filename=smart_open-3.0.0-py3-none-any.whl size=107097 sha256=d9b8bc294d38ce5cfbd5f7f833f7d664da8afac83ee42afdaa32a776c35bb4d1
  Stored in directory: /home/ec2-user/.cache/pip/wheels/88/2a/d4/f2e9023989d4d4b3574f268657cb6cd23994665a038803f547
Successfully built smart-open


In [2]:
import boto3
from trp import Document, Page, BoundingBox
from PIL import Image, ImageDraw, ImageFont
from smart_open import open
from sagemaker.session import Session
import minecart
import pandas as pd
%matplotlib inline

Matplotlib is building the font cache using fc-list. This may take a moment.


In [15]:
# initiate s3 bucket and corresponding data folder
bucket = "ran-s3-systemic-risk"
data_folder ="X17A5/"
# balance_sheet_idx = 3  # the page of the report you care about

# Amazon Textract client and Sagemaker session
textract = boto3.client('textract')
session = Session()

# discover all of the pdfs that you want to parse
paths = session.list_s3_files(bucket, data_folder)
test_key = paths[1]

In [16]:
test_key

'X17A5/280 SECURITIES LLC18.pdf'

In [5]:
def textract_from_s3(bucket_name, key, textract_client=textract) -> Document:
    """
    Gets an analyzed document from textract from an initialized client
    :param bucket_name:
    :param key:
    :param textract_client:
    :return:
    """
    r = textract_client.analyze_document(
            Document={
                'S3Object': {
                    'Bucket': bucket_name,
                    'Name': key
                }
            },
            FeatureTypes=["TABLES"])

    return Document(r)

def get_image_from_pdf(target_pdf_path, page_num=0, image_num=0) -> Image:
    """
    Gets a single image from a pdf, given a page and image position
    :param target_pdf_path:
    :param page_num:
    :param image_num:
    :return:
    """
    with open(target_pdf_path, 'rb') as f:
        try:
            image = minecart.Document(f).get_page(page_num).images[image_num].as_pil()
            return image
        except UnicodeError or IOError as e:
            print("issue getting the image")


def draw_bbox(bbox: BoundingBox, img_width, img_height):
    """
    using ImageDraw, draw a red outline around a Cell's boundingbox
    :param bbox:
    :param img_width:
    :param img_height:
    :return:
    """
    x1 = bbox.left * img_width
    y1 = bbox.top * img_height-2
    x2 = x1 + (bbox.width * img_width) + 5
    y2 = y1 + (bbox.height * img_height) + 2

    draw = ImageDraw.Draw(img)
    draw.rectangle([x1, y1, x2, y2], fill=None, outline=(255, 0, 0, 100))
    draw.text((x1, y1), text, fill="blue", font=ImageFont.load_default())


def parse_balance_sheet(document_page:Page, columns=("YEAR1", "YEAR2")) -> pd.DataFrame:
    """
    This is your function for parsing a certain page of interest
    :param document_page:
    :param columns:
    :return:
    """
    rows = []  # list of lists [[cell1, cell2,...]]
    for table in document_page.tables:
        for r, row in enumerate(table.rows):
            assert len(row.cells) == columns, "The length of column names and the actual columns don't match"
            data = []
            for c, cell in enumerate(row.cells):
                print("Table[{}][{}] = {}".format(r, c, cell.text))
                data.append(cell.text)
                if c == 0:
                    item_name = cell.text
                    print(r, "FIRST CELL ", item_name)
            rows.append(data)

    df = pd.DataFrame(rows, columns=columns)

    return df


In [21]:
import time

In [22]:
# content modified from Amazon AWS Textract repository (refer to URL below)
# https://github.com/aws-samples/amazon-textract-code-samples/blob/master/python/12-pdf-text.py

def startJob(s3BucketName:str, objectName:str) -> str:
    """
    Starts a Textract job on AWS server 
    """
    response = None
    client = boto3.client('textract')
    
    # issue response to AWS to start Textract job
    response = client.start_document_text_detection(
            DocumentLocation={
                'S3Object': {
                    'Bucket': s3BucketName,
                    'Name': objectName
                }
            })
    
    # return response job ID for service
    return response["JobId"]

def isJobComplete(jobId:str) -> str:
    """
    Tracks the completion status of the Textract job when qued
    """
    time.sleep(5)
    client = boto3.client('textract')
    response = client.get_document_text_detection(JobId=jobId)
    status = response["JobStatus"]
    print("Job status: {}".format(status))
    
    # check current status of AWS job (ask server every 5 seconds for data)
    while(status == "IN_PROGRESS"):
        time.sleep(5)                   # lag before reporting status
        response = client.get_document_text_detection(JobId=jobId)
        status = response["JobStatus"]
        print("Job status: {}".format(status))

    return status

def getJobResults(jobId:str) -> list:
    """
    Returns the contents of the Textract job, after completion status met
    """
    pages = []          # initialize list object to track pages

    client = boto3.client('textract')
    response = client.get_document_text_detection(JobId=jobId)
    
    pages.append(response)
    print("Resultset page recieved: {}".format(len(pages)))
    
    nextToken = None
    if('NextToken' in response):
        nextToken = response['NextToken']
    
    # iterate through the pages and append to response figure
    while(nextToken):

        response = client.get_document_text_detection(JobId=jobId, 
                                                      NextToken=nextToken)

        pages.append(response)
        print("Resultset page recieved: {}".format(len(pages)))
        nextToken = None
        if('NextToken' in response):
            nextToken = response['NextToken']

    return pages

In [25]:
# S3 storage for files on AWS site    
jobId = startJob(bucket, test_key)
print("Started job with id: {}".format(jobId))
if(isJobComplete(jobId)):
    response = getJobResults(jobId)

Started job with id: 22382d0fdc55db0dc511bd5f069c63b204e961da5b8b27e41455c7188dc7ee6f
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
Resultset page recieved: 4
[{'DocumentMetadata': {'Pages': 12}, 'JobStatus': 'SUCCEEDED', 'NextToken': 'ab/gaT8idyc+mge0Av8FZe+lmUpTnlV1tZ/jW354VMb8cENSLUcfTlpMSwOWswSAL6gciWkjw1O3E6jB31CEOhcYb/rm042rPmIz0tulYE7GhcloJQ==', 'Blocks': [{'BlockType': 'PAGE', 'Geometry': {'BoundingBox': {'Width': 1.0, 'Height': 1.0, 'Left': 0.0, 'Top': 0.0}, 'Polygon': [{'X': 0.0, 'Y': 0.0}, {'X': 1.0, 'Y': 0.0}, {'X': 1.0, 'Y': 1.0}, {'X': 0.0, 'Y': 1.0}]}, 'Id': '5b8009e6-ed0b-4321-8a3b-c96d88546b06', 'Relationships': [{'Type': 'CHILD', 'Ids': ['e226b460-f314-

In [31]:
# Print detected text from PDF files
for resultPage in response:
    for item in resultPage["Blocks"]:
        if item["BlockType"] == "LINE":
            print(item['Text'])
#             print ('\033[94m' +  item["Text"] + '\033[0m')

UNITEDSTATES
OMB APPROVAL
SECURITIESANDEXC EXCHANGE COMMISSION
OMB Number:
3235-0123
Washington, D.C. 20549
Expires:
August 31, 2020
Estimated average burden
ANNUAL AUDITED REPORT
hours per response
12.00
FORM X-17A-5
SEC FILE NUMBER
PART III
-69819
FACING PAGE
Information Required of Brokers and Dealers Pursuant to Section 17 of the
Securities Exchange Act of 1934 and Rule 17a-5 Thereunder
REPORT FOR THE PERIOD BEGINNING
01/01/17
AND ENDING
12/31/17
MM/DD/YY
MM/DD/YY
A. REGISTRANT IDENTIFICATION
NAME OF BROKER-DEALER: 280 Securities LLC
OFFICIAL USE ONLY
ADDRESS OF PRINCIPAL PLACE OF BUSINESS: (Do not use P.O. Box No.)
FIRM I.D. NO.
575 Market Street, Suite 3800
(No. and Street)
San Francisco
CA
94105
(City)
(State)
(Zip Code)
NAME AND TELEPHONE NUMBER OF PERSON TO CONTACT IN REGARD TO THIS REPORT
Heather Hall
628-231-2493
(Area Code - Telephone Number)
B. ACCOUNTANT IDENTIFICATION
INDEPENDENT PUBLIC ACCOUNTANT whose opinion is contained in this Report*
Moss Adams LLP
(Name - if indiv

**Points made on course of project**
- Text data seems to be read easily from PDF, but table data seems to be better suited for balance sheet figures but not primary figures (1st page filings may differ for newer firms or entries)
    - secure a list of figures to extract potentially (what are all the data points we want)
- SEC Parser (how can we optimize the existing code base - walkthrough script) 
- Name of firm, CIK

In [24]:
# Call Amazon Textract - you can call this from a function,
# docs are here: https://docs.aws.amazon.com/textract/latest/dg/what-is.html
response = textract.analyze_document(
    Document={
        'S3Object': {
            'Bucket': bucket,
            'Name': test_key
        }
    },
    FeatureTypes=["TABLES"])

doc = Document(response)
print(doc)
# then do something with the response

UnsupportedDocumentException: An error occurred (UnsupportedDocumentException) when calling the AnalyzeDocument operation: Request has unsupported document format

In [ ]:
# Work in Progress, get an image.
# Minecart doesn't seem to work on this kernel, so commenting it out
# test_path = f's3://{bucket}/{test_key}'
# img = get_image_from_pdf(test_path, page_num=balance_sheet_idx, image_num=0)
# width, height = img.size
# print(width, height)
# # save a copy for later
# img.save(f'{test_key}-{balance_sheet_idx}.jpg')

In [17]:
# this is how to iterate over the Document
# if you have an image open, you can draw the bounding boxes over it

for i, page in enumerate(doc.pages):
    # optional: filter for a page you care about
    if i != balance_sheet_idx:
        continue

    # You could pass this page into a function if you know how to parse it
    # df = parse_balance_sheet(page, columns=["mycolumn-one", "mycolumn-two"])

    # Print tables' cells
    for table in page.tables:
        for r, row in enumerate(table.rows):
            print(row)
            for c, cell in enumerate(row.cells):
                print("Table[{}][{}] = {}".format(r, c, cell.text))
                text = cell.text
                # do something with the text

                # optionally, draw a bounding box around a cell
                # geo: Geometry = cell.geometry
                # bbox: BoundingBox = geo.boundingBox
                # draw_bbox(bbox, img_width=width, img_height=height)

# save the newly annotated image
# img.save("bboxed-test_image.jpg")

[][AS OF ][MARCH 31, ]
[][2006 ][2005 ]
[ASSETS: ][][]
[Cash and cash equivalents ][$ 14,652 ][$ 19,543 ]
[Commissions receivable ][15,124 ][20,749 ]
[TOTAL ASSETS ][$ 29,776 ][S$ 40,292 ]
[LIABILITIES AND STOCKHOLDER'S ][EQUITY ][]
[LIABILITIES: ][][]
[Commissions payable ][$ 12,099 ][$ 16,599 ]
[Total liabilities ][12,099 ][16.599 ]
[STOCKHOLDER'S EQUITY: ][][]
[Common stock, $.01 par value; 1,000 shares authorized, 1,000 shares issued and outstanding ][10 ][10 ]
[Additional paid-in capital ][9,990 ][9,990 ]
[Retained earnings ][7,677 ][13.693 ]
[Total stockholder's equity ][17,677 ][23.693 ]
[TOTAL LIABILITIES AND STOCKHOLDER'S EQUITY ][$ 29,776 ][S 40,292 ]


In [ ]:
# Call Amazon Textract on a set of paths and do something with it
for path in paths:

    doc = textract_from_s3(bucket, path)
    # parse page table into csv, if we have a template for it, otherwise print the table
    for i, page in enumerate(doc.pages):
        if i == balance_sheet_idx:
            df = parse_balance_sheet(page)
            df.to_csv(f"{path}-{i}.csv")
        else:
            print("skipping page", i)

